<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/experiment/%E6%94%B9%E5%96%84%E5%BE%8C%E6%A8%A1%E7%B5%84%E6%BA%AB%E5%BA%A6%E5%8F%8A%E8%8B%AF%E7%9A%84%E9%97%9C%E4%BF%82(%E7%A2%BA%E4%BF%9D%E6%98%AF%E8%B2%A0%E7%9B%B8%E9%97%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt
!pip install optuna > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 2292, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 2292 (delta 7), reused 0 (delta 0), pack-reused 2270
Receiving objects: 100% (2292/2292), 63.68 MiB | 23.37 MiB/s, done.
Resolving deltas: 100% (1500/1500), done.


# import

In [2]:
from c620.FV2 import AllSystem
from c620.configV2 import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


# demo data

In [3]:
demo = joblib.load('./data/demo.pkl')
print(demo.keys())
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
icg_input.head()

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c620_wt', 'c660_op', 'c660_wt', 'c670_op', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
118-017,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,72.0,0.2127,35.413666,57.973106,80.0,920.0,0.01,5.000005,36.0


# 這一筆樣本的NA in Benzene_ppmw只有920 期望能拉到 980

In [4]:
icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
118-017,920.0


# 期望能從80降到70

In [5]:
icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
118-017,80.0


# 期望能從5拉到10

In [6]:
icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
118-017,5.000005


# 1.試算模式

In [7]:
f = joblib.load('/content/c620/model/allsystem.pkl')

In [8]:
demo['icg_input']

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
118-017,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,72.0,0.2127,35.413666,57.973106,80.0,920.0,0.01,5.000005,36.0


# 確認溫度跟苯是負相關

In [9]:
table = pd.DataFrame()
for i in [70,80,90]:
  icg_input = demo['icg_input']
  icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'] = i 
  c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f.inference(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])
  table = table.append(c620_op.iloc[:,-2:].join(icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']))
table

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
118-017,178.017532,193.112747,70
118-017,177.337234,192.644394,80
118-017,175.516098,192.260590,90


# 推薦模式

In [10]:
f = joblib.load('/content/c620/model/allsystem.pkl')

# 確認溫度跟苯是負相關

In [15]:
from tqdm import tqdm_notebook as tqdm
table = pd.DataFrame()
for i in [70,80,90]:
  c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op,_,_,_ = f.recommend(demo['icg_input'],demo['c620_feed'],demo['t651_feed'],auto_set_recommended_value=[i,980,10])
  table = table.append(c620_op.iloc[:,-2:].join(icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']))
table

[I 2021-05-01 08:54:04,269] A new study created in memory with name: no-name-7e07a161-53b5-4a69-a688-ffb39d09408d


系統已經自動設定推薦值[70,980,10]



bzinside_loss: 0.06290592696516625
nainbz_loss: 0.0460433666557956
tol_loss: 3.693099857648125
distrate_loss: 0


[I 2021-05-01 08:55:06,216] A new study created in memory with name: no-name-7e7a7d9c-3ec1-45dd-bc98-aa2d8d36cb50


系統已經自動設定推薦值[80,980,10]



bzinside_loss: 0.12942836083876869
nainbz_loss: 0.04456007875549424
tol_loss: 2.4817856028846434
distrate_loss: 0


[I 2021-05-01 08:56:07,724] A new study created in memory with name: no-name-725a2666-c1de-4aef-aeab-dea1ee4ba2c3


系統已經自動設定推薦值[90,980,10]



bzinside_loss: 0.06645092480261923
nainbz_loss: 0.028533772328097833
tol_loss: 2.309321390244346
distrate_loss: 0


,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
118-017,178.120758,194.744507,70
118-017,177.917532,193.012747,80
118-017,177.237234,192.544394,90
